In [ ]:
!pip install kaggle

In [9]:
!KAGGLE_USERNAME=datadinosaur KAGGLE_KEY=xxxxxxxxxxxxxx kaggle datasets download -d eivankin/s2-naip-5k-pairs-rgb-nir

t


In [1]:
!git clone https://github.com/eivankin/satlas-SR-location-embeddings

Cloning into 'satlas-SR-location-embeddings'...
remote: Enumerating objects: 381, done.
remote: Counting objects: 100% (376/376), done.
remote: Compressing objects: 100% (212/212), done.
remote: Total 381 (delta 217), reused 291 (delta 163), pack-reused 5 (from 1)
Receiving objects: 100% (381/381), 18.37 MiB | 34.52 MiB/s, done.
Resolving deltas: 100% (217/217), done.


In [13]:
!cd satlas-SR-location-embeddings && git pull 

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5 (delta 3), reused 5 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 734 bytes | 367.00 KiB/s, done.
From https://github.com/eivankin/satlas-SR-location-embeddings
   d98d0bf..83eac28  main       -> origin/main
Updating d98d0bf..83eac28
Fast-forward
 ssr/data/s2-naip_dataset.py | 21 +++++++++++++--------
 1 file changed, 13 insertions(+), 8 deletions(-)


In [3]:
!cd satlas-SR-location-embeddings && mkdir weights && cd weights && wget https://pub-25c498004d1e4d4c8da69b2c05676836.r2.dev/esrgan_1S2.pth

--2025-05-25 16:02:07--  https://pub-25c498004d1e4d4c8da69b2c05676836.r2.dev/esrgan_1S2.pth
Resolving pub-25c498004d1e4d4c8da69b2c05676836.r2.dev (pub-25c498004d1e4d4c8da69b2c05676836.r2.dev)... 162.159.140.237, 172.66.0.235, 2606:4700:7::eb, ...
Connecting to pub-25c498004d1e4d4c8da69b2c05676836.r2.dev (pub-25c498004d1e4d4c8da69b2c05676836.r2.dev)|162.159.140.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 134060750 (128M) [application/zip]
Saving to: ‘esrgan_1S2.pth’

esrgan_1S2.pth      100%[===================>] 127.85M  75.0MB/s    in 1.7s    

2025-05-25 16:02:09 (75.0 MB/s) - ‘esrgan_1S2.pth’ saved [134060750/134060750]



In [11]:
%%writefile satlas-SR-location-embeddings/ssr/options/osm_obj_esrgan.yml

# General Settings
name: osm-obj-esrgan-nir
model_type: OSMObjESRGANModel
scale: 4
num_gpu: auto
manual_seed: 0

# USM the Ground-Truth
l1_gt_usm: True
percep_gt_usm: True
gan_gt_usm: False

# Feed the discriminator the same low-res images as the generator receives
feed_disc_lr: True

# Weight for OSM object loss within GAN loss calculation
osm_obj_weight: 0.3

# Dataset and Dataloader Settings
datasets:
  train:
    name: train
    type: S2NAIPDataset

    sentinel2_path: s2-naip-5k-pairs-rgb-nir/train/sentinel2
    naip_path: s2-naip-5k-pairs-rgb-nir/train/naip

#    tile_weights: super_resolution_2023-12-08/train_tile_weights/urban_set_weights.json

    osm_objs_path: s2-naip-5k-pairs-rgb-nir/train/openstreetmap

    # Min number of OSM features per tile to be used. 
    # NOTE: Number of samples is currently clipped to this number, which reduces dataset size.
    n_osm_objs: 1

    use_shuffle: True #False  # shuffle must be false if using tile_weights
    rand_crop: True  # random cropping and resizing augmentation (training only)

    plot_inputs: False

    num_worker_per_gpu: 1
    batch_size_per_gpu: 8

    n_s2_images: 1

    s2_bands: [7, 6, 5] # [39, 38, 37] for RGB of q_18 merge, [35, 34, 33] for RGB for q_50, [40, 39, 38] for NIR,R,G for q_18
    naip_bands: [0, 1, 2] # [0, 1, 2] for RGB, [3, 0, 1] NIR,R,G

    io_backend:
      type: disk

  val:
     name: validation
     type: S2NAIPDataset

     sentinel2_path: s2-naip-5k-pairs-rgb-nir/val/sentinel2
     naip_path: s2-naip-5k-pairs-rgb-nir/val/naip

     use_shuffle: False

     n_s2_images: 1

     s2_bands: [7, 6, 5] # [39, 38, 37] for RGB of q_18 merge, [35, 34, 33] for RGB for q_50, [40, 39, 38] for NIR,R,G for q_18
     naip_bands: [0, 1, 2] # [0, 1, 2] for RGB, [3, 0, 1] NIR,R,G

     io_backend:
       type: disk

test_datasets:
   test:
     name: test
     type: S2NAIPDataset
     phase: test
     scale: 4

     sentinel2_path: s2-naip-5k-pairs-rgb-nir/test/sentinel2
     naip_path: s2-naip-5k-pairs-rgb-nir/test/naip

     use_shuffle: False

     s2_bands: [7, 6, 5] # [39, 38, 37] for RGB of q_18 merge, [35, 34, 33] for RGB for q_50, [40, 39, 38] for NIR,R,G for q_18
     naip_bands: [0, 1, 2] # [0, 1, 2] for RGB, [3, 0, 1] NIR,R,G

     n_s2_images: 1

     io_backend:
       type: disk

# Network Structures
network_g:
  type: SSR_RRDBNet
  num_in_ch: 3  # number of Sentinel2 images * 3 channels (RGB)
  num_out_ch: 3
  num_feat: 64
  num_block: 23
  num_grow_ch: 32

network_d:
  type: OSMObjDiscriminator
  num_in_ch: 6  # RGB of Real/Fake image + [optional] old high-res image + [optional] low-res images
  num_feat: 64
  skip_connection: True

# Load in existing weights to the generator and discriminator
# Uncomment pretrain_network_g and pretrain_network_d and add paths to your weights
path:
  pretrain_network_g: weights/esrgan_1S2.pth
  param_key_g: params_ema
  strict_load_g: false
#  pretrain_network_d: weights/net_d_pre.pth
  param_key_d: params
  strict_load_d: true
  resume_state: ~
  visualization: .

# Training Settings
train:
  ema_decay: 0.999
  optim_g:
    type: Adam
    lr: !!float 1e-4
    weight_decay: 0
    betas: [0.9, 0.99]
  optim_d:
    type: Adam
    lr: !!float 1e-4
    weight_decay: 0
    betas: [0.9, 0.99]

  scheduler:
    type: MultiStepLR
    milestones: [10000]
    gamma: 0.5

  total_iter: 12000
  warmup_iter: -1  # no warm up todo: warmup where only missing weights (projection, attention?) is trained

  # Losses
  pixel_opt:
    type: L1Loss
    loss_weight: 1.0
    reduction: mean
  # Perceptual Loss (content and style losses)
  perceptual_opt:
    type: PerceptualLoss
    layer_weights:
      # before relu
      'conv1_2': 0.1
      'conv2_2': 0.1
      'conv3_4': 1
      'conv4_4': 1
      'conv5_4': 1
    vgg_type: vgg19
    use_input_norm: true
    perceptual_weight: !!float 1.0
    style_weight: 0
    range_norm: false
    criterion: l1
  # GAN Loss
  gan_opt:
    type: GANLoss
    gan_type: vanilla
    real_label_val: 1.0
    fake_label_val: 0.0
    loss_weight: !!float 1e-1
  # CLIP similarity loss
  clip_opt:
    type: CLIPLoss
    clip_loss_model: ViT-B-16-SigLIP-256
  # TODO: location matching loss

  net_d_iters: 1
  net_d_init_iters: 0

# Validation Settings
val:
   val_freq: !!float 5e2
   save_img: True
   pbar: True

   metrics:
     psnr:
       type: calculate_psnr
       crop_border: 4
       test_y_channel: false
     ssim:
       type: calculate_ssim
       crop_border: 4
       test_y_channel: false
     cpsnr:
       type: calculate_cpsnr
       crop_border: 4
       test_y_channel: false
#     lpips:
#       type: calculate_lpips
#       lpips_model: vgg
     clipscore:
       type: calculate_clipscore
       clip_model: clip-ViT-B/16  # see ssr/metrics/clipscore.py for currently supported models

# Testing Settings
test:
  save_img: True

  metrics:
     psnr:
       type: calculate_psnr
       crop_border: 4
       test_y_channel: false
     ssim:
       type: calculate_ssim
       crop_border: 4
       test_y_channel: false
     cpsnr:
       type: calculate_cpsnr
       crop_border: 4
       test_y_channel: false
     lpips:
       type: calculate_lpips
       lpips_model: vgg
     clipscore:
       type: calculate_clipscore
       clip_model: siglip-ViT-SO400M-14  # see ssr/metrics/clipscore.py for currently supported models

# Logging Settings
logger:
  print_freq: 10
  save_checkpoint_freq: !!float 5e2
  use_tb_logger: true
  wandb:
    project: multisat
    resume_id: ~

# Dist Training Settings
dist_params:
  backend: nccl
  port: 29500


Overwriting satlas-SR-location-embeddings/ssr/options/osm_obj_esrgan.yml


In [5]:
!pip install -qqq -r satlas-SR-location-embeddings/requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 53.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56

In [6]:
!pip install -qqq rasterio transformers==4.33.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 79.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 78.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.5 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.33.3 which is incompatible.


In [7]:
import wandb
wandb.login(key="7283680b6598956ca669e7b22d0abe0a557a47aa")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: eivankin (eivankin-innopolis-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
!cd satlas-SR-location-embeddings && python -m ssr.train -opt ssr/options/osm_obj_esrgan.yml      

/usr/local/lib/python3.11/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
Disable distributed.
Path already exists. Rename it to /kaggle/working/satlas-SR-location-embeddings/experiments/osm-obj-esrgan-nir_archived_20250525_164416
Path already exists. Rename it to /kaggle/working/satlas-SR-location-embeddings/tb_logger/osm-obj-esrgan-nir_archived_20250525_164416
2025-05-25 16:44:16,793 INFO: 
                ____                _       _____  ____
               / __ ) ____ _ _____ (_)_____/ ___/ / __ \
              / __  |/ __ `// ___// // ___/\__ \ / /_/ /
             / /_/ // /_/ /(__  )/ // /__ ___/ // _, _/
            /_____/ \__,_//____//_/ \___//____//_/ |_|
     ______                   __